In [14]:
import telebot
from telebot import types


from bs4 import BeautifulSoup
import numpy as np
from fake_useragent import UserAgent
import warnings
from itertools import chain
import pymorphy2
import requests
import PyPDF2
from io import BytesIO

warnings.filterwarnings("ignore")

In [15]:
# хотфикс для версии Python >= 3.11
def pymorphy2_311_hotfix():
    from inspect import getfullargspec
    from pymorphy2.units.base import BaseAnalyzerUnit

    def _get_param_names_311(klass):
        if klass.__init__ is object.__init__:
            return []
        args = getfullargspec(klass.__init__).args
        return sorted(args[1:])

    setattr(BaseAnalyzerUnit, '_get_param_names', _get_param_names_311)

def get_gent_form(word):
    pymorphy2_311_hotfix()
    morph = pymorphy2.MorphAnalyzer()
    parsed_word = morph.parse(word.lower())[0]
    if parsed_word.tag.number == 'sing':
        return parsed_word.inflect({'gent'}).word
    else:
        return parsed_word.inflect({'gent', 'plur'}).word

In [27]:
# сейчас сбор идет по первой странице start=0, TBD по N первым важным страницам
def get_data(word):    
    id = 0
    query = f'безопасность+{get_gent_form(word[4:].strip())}'
    print(query)
    page_link = f'https://scholar.google.com/scholar?start={id}&q={query}&hl=ru&as_sdt=0,5'
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        return
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    results = []
    for item in soup.select('.gs_ri'):
        title = item.select_one('.gs_rt a').text if item.select_one('.gs_rt a') else 'Нет заголовка'
        link = item.select_one('.gs_rt a')['href'] if item.select_one('.gs_rt a') else 'Нет ссылки'
        snippet = item.select_one('.gs_rs').text if item.select_one('.gs_rs') else 'Нет аннотации'
        
        results.append({
            'title': title,
            'link': link,
            'snippet': snippet
        })

    obj = soup.findAll(lambda tag: tag.name == 'div')
    page_links = [link.find('a')['href'] for link in obj if link.find('a')]
    page_links_unique = list(set(page_links))

    return results

In [18]:
def check_link(url):
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    return response.ok

def get_data_by_link(url):    
    page_link = url
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        return
    
    if url[-3:] == 'pdf':
        cnt_pages = 0
        with BytesIO(response.content) as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ''
            for page in reader.pages:
                cnt_pages += 1
                text += page.extract_text() + '\n'
            print(f"done {cnt_pages} pages!")
            return text
        
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    paragraphs = soup.find_all('p')
    article_text = ' '.join([para.get_text() for para in paragraphs])
    
    return article_text

In [20]:
def collect_links(info):
    result = []
    for item in info:
        result.append(item['link'])
    return result

In [21]:
with open('key.txt') as f:
    TOKEN = f.read().strip()
TOKEN 

'8317129367:AAGGO8pYnbQHZCe7Vx8yZb0Wtux9MAvbaqU'

In [30]:
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=['start'])
def send_welcome(message):
    welcome_text = (
        "Добро пожаловать в бот для оценки безопасности химических веществ!\n"
        "Вы можете вводить названия химических веществ, и я предоставлю информацию о их безопасности.\n"
        "Примеры веществ: вода, ацетон, бензин, спирт, хлор, аммиак.\n"
        "Введите название химического вещества для получения информации или подборки статей."
    )
    markup = types.InlineKeyboardMarkup(row_width=2)
    button3 = types.InlineKeyboardButton(text="Подбор статей о веществе", callback_data="articles")
    button4 = types.InlineKeyboardButton(text="Спроси LLM о веществе", callback_data="llm")
    markup.add(button3, button4)
    bot.send_message(message.chat.id, welcome_text, reply_markup=markup)

@bot.callback_query_handler(func=lambda x: x.data)
def start_message(callback):
    if callback.data == "llm":
        llm_message = "Введи название вещества начав предложение с LLM:"
        bot.send_message(callback.message.chat.id, llm_message)
    elif callback.data == "articles":
        eqt_message = "Введи название вещества начав предложение с ART:"
        bot.send_message(callback.message.chat.id, eqt_message)

@bot.message_handler(func=lambda x: x.text.startswith("LLM"))
def llm_request(message):
    substance = message.text.lower()
    # safety_info = safety_data.get(substance, "Извините, у меня нет информации об этом веществе.")
    safety_info = "Безопасно."
    bot.send_message(message.chat.id, safety_info)

@bot.message_handler(func=lambda x: x.text.startswith("ART"))
def cr_request(message):
    equation = message.text
    result = get_data(equation)
    links = collect_links(result)
    message_text = "\n".join(links)
    bot.send_message(message.chat.id, message_text)

bot.infinity_polling()

безопасность+воды


2025-08-07 15:09:41,386 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2025-08-07 15:09:41,388 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
